In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

In [4]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [5]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

In [6]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)


In [7]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

291/291 - 39s - loss: 0.8291 - accuracy: 0.6450
144/144 - 2s - loss: 0.7625 - accuracy: 0.6656
0.7624974846839905
0.6655744910240173
['loss', 'accuracy']


In [28]:
from keras.engine.saving import load_model
model.save('dlicp5.h5')

In [29]:
model1 = load_model('dlicp5.h5')

In [30]:
import numpy as np
sentences = np.array(["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"])

prediction=tokenizer.texts_to_sequences(sentences)
prediction=pad_sequences(prediction,maxlen=28)
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model1.predict(prediction))

[[0.93990195 0.04176805 0.01833006]]


In [23]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [12]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
from keras.wrappers.scikit_learn import KerasClassifier
my_model = KerasClassifier(build_fn=createmodel,verbose=0)
batch_size = [32,64]
epochs = [1,2]
param_grid = dict(batch_size=batch_size,epochs=epochs)

In [13]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=my_model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, Y_train)

In [14]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.679220 using {'batch_size': 32, 'epochs': 2}


In [16]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.engine.saving import load_model
from sklearn.preprocessing import LabelEncoder
import numpy as np

data = pd.read_csv('spam.csv',encoding="latin-1")

data = data[['v2','v1']]
print(data)
data['v2'] = data['v2'].apply(lambda x: x.lower())
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)
max=0

for i in X:
  if len(i)>max:
    max=len(i)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model1 = Sequential()
    model1.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model1.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model1.add(Dense(2,activation='softmax'))
    model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model1
print(model.summary())

                                                     v2    v1
0     Go until jurong point, crazy.. Available only ...   ham
1                         Ok lar... Joking wif u oni...   ham
2     Free entry in 2 a wkly comp to win FA Cup fina...  spam
3     U dun say so early hor... U c already then say...   ham
4     Nah I don't think he goes to usf, he lives aro...   ham
...                                                 ...   ...
5567  This is the 2nd time we have tried 2 contact u...  spam
5568              Will Ì_ b going to esplanade fr home?   ham
5569  Pity, * was in mood for that. So...any other s...   ham
5570  The guy did some bitching but I acted like i'd...   ham
5571                         Rofl. Its true to its name   ham

[5572 rows x 2 columns]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 128)           256000    
_____________

In [17]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model1 = createmodel()
model1.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

117/117 - 115s - loss: 0.1704 - accuracy: 0.9440


In [18]:
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)
model1.save('dlicp5.h5')
model1 = load_model('dlicp5.h5')

58/58 - 16s - loss: 0.0689 - accuracy: 0.9821
0.06890879571437836
0.9820554852485657
['loss', 'accuracy']
